In [1]:
import os
import sys
import yaml
import rclpy
import random
import numpy as np
from geometry_msgs.msg import PoseStamped, Pose
from moveit.planning import MoveItPy
from moveit.core.robot_state import RobotState
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory
from rclpy.node import Node
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint

# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(robot_name="panda", package_name="moveit_resources_panda_moveit_config")
        .robot_description(file_path="config/panda.urdf.xacro")
        .trajectory_execution(file_path="config/gripper_moveit_controllers.yaml")
        .moveit_cpp(
            file_path=os.path.join(
                get_package_share_directory("moveit2_tutorials"),
                "config",
                "jupyter_notebook_prototyping.yaml"
        )
    )
    .to_moveit_configs()
    ).to_dict()

In [2]:
# initialise rclpy (only for logging purposes)
rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
panda = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
panda_arm = panda.get_planning_component("panda_arm")
panda_gripper = panda.get_planning_component("hand")

[INFO] [1719044930.660967635] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1719044930.661607961] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1719044930.733632694] [moveit_cpp_initializer]: params_filepath: /tmp/launch_params_l37y8zsz
[INFO] [1719044930.733713494] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1719044930.959551319] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1719044930.963169161] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.00342717 seconds
[INFO] [1719044930.963201127] [moveit_robot_model.robot_model]: Loading robot model 'panda'...
[INFO] [1719044931.019149417] [moveit_kinematics_base.kinematics_base]: Joint weights for group 'panda_arm': 1 1 1 1 1 1 1
[INFO] [1719044931.049581449] [moveit.ros_planning_interface.moveit_cpp]: Listening to '/joint_states' for joint states
[INFO] [1719044931.052932195] [moveit_ros.current_state_monitor]: Listening to joint states on topic '/joint_states'
[INFO] [1719044931.057

In [5]:
def plan_and_execute(
    robot,
    planning_component,
    single_plan_parameters=None,
    multi_plan_parameters=None,
):
    """A helper function to plan and execute a motion."""
    # plan to goal
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        print("Planning failed")
        


# 특정 좌표로 이동시키는 함수
def move_to_pose(robot, planning_component, pose):
    planning_component.set_start_state_to_current_state()
    planning_component.set_goal_state(pose_stamped_msg=pose, pose_link="panda_link8")
    plan_and_execute(robot, planning_component)

# 그리퍼를 여는 함수 (가상의 함수로 대체)
def open_gripper():
    panda_gripper.set_start_state_to_current_state()
    panda_gripper.set_goal_state("open")
    plan_and_execute(panda, panda_gripper)
    print("Gripper opened")

# 그리퍼를 닫는 함수 (가상의 함수로 대체)
def close_gripper():
    panda_gripper.set_start_state_to_current_state()
    panda_gripper.set_goal_state("close")
    plan_and_execute(panda, panda_gripper)
    print("Gripper closed")
   
    
place_pose_xy = [(-0.2, -0.2), (-0.25, -0.2),(-0.2, -0.25),(-0.25,-0.25),(-0.2,-0.3),(-0.25,-0.3)]

# 초기 위치 정의 (특정 좌표로 설정)
initial_pose = PoseStamped()
initial_pose.header.frame_id = "panda_link0"
initial_pose.pose.orientation.w = 0.0
initial_pose.pose.orientation.x = 0.0
initial_pose.pose.orientation.y = 1.0
initial_pose.pose.orientation.z = 0.0
initial_pose.pose.position.x = 0.3
initial_pose.pose.position.y = 0.3
initial_pose.pose.position.z = 0.7

# 목표 좌표 정의
pick_pose = PoseStamped()
pick_pose.header.frame_id = "panda_link0"
pick_pose.pose.orientation.w = 0.0
pick_pose.pose.orientation.z = 0.0
pick_pose.pose.orientation.y = 1.0
pick_pose.pose.orientation.z = 0.0
#pick_pose.pose.position.x = -0.28
#pick_pose.pose.position.y = -0.2
pick_pose.pose.position.z = 0.1

place_pose = PoseStamped()
place_pose.header.frame_id = "panda_link0"
place_pose.pose.orientation.w = 0.0
place_pose.pose.orientation.x = 0.0
place_pose.pose.orientation.y = 1.0
place_pose.pose.orientation.z = 0.0
#place_pose.pose.position.x = 0.5
#place_pose.pose.position.y = 0.0
place_pose.pose.position.z = 0.3


for i in range(6):
    # 1. 초기 위치로 이동
    move_to_pose(panda, panda_arm, initial_pose)

    # 2. 특정 위치로 이동
    x = random.uniform(0.0, 0.5)  # Random x position
    y = random.uniform(0.0, 0.5)  # Random y position

    pick_pose.pose.position.x = x
    pick_pose.pose.position.y = y
    move_to_pose(panda, panda_arm, pick_pose)

    # 3. gripper 통해 객체 잡기 (구현x)
    open_gripper()

    close_gripper()

    x, y = place_pose_xy[i]
    place_pose.pose.position.x = x
    place_pose.pose.position.y = y
    
    # 4. 객체를 특정 위치로 이동
    move_to_pose(panda, panda_arm, place_pose)
    
    # 5. 객체 놓기
    open_gripper()


move_to_pose(panda, panda_arm, initial_pose)

[WARN] [1719045259.434156467] [moveit_py]: Parameter 'plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file


Gripper opened
Gripper closed


[WARN] [1719045259.434318127] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1719045259.435735306] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1719045259.551449068] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045259.551500637] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045259.551543013] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045259.551554266] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045259.551585236] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01

Gripper opened


ning_context]: Planner configuration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1719045263.453391981] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045263.453434791] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045263.453474789] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045263.453485986] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045263.453516652] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1719045263.462243150] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1719045263.462324055] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045263.462357014] [m

...

onfiguration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1719045290.219659587] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045290.219726651] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045290.219792024] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045290.219812579] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045290.219866407] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1719045290.221869399] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1719045290.221963075] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045290.221999580] [moveit.plugins.moveit_sim

Gripper opened


]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1719045291.653652234] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'hand' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1719045291.691063309] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045291.691141037] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045291.691184755] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045291.691209477] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045291.691243325] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1719045291.702083856] [moveit_ros.tr

Gripper opened
Gripper closed
Gripper opened


O] [1719045292.037301629] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045292.037345701] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1719045292.041882030] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1719045292.041947102] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045292.041976357] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1719045292.042142654] [moveit.simple_controller_manager.follow_joint_trajectory_controller_handle]: sending trajectory to panda_arm_controller
[INFO] [1719045292.043304949] [moveit.simple_controller_manager.follow_joint_trajectory_controller_handle]: panda_arm_controller started execution
[INFO] [1719045292.043338785] [moveit.simple_controller_manager.follow_joint_trajectory_controller_handle]: Goal request accepted!
[IN

In [ ]:
rclpy.shutdown()